In [1]:
import spacy
import nltk.data
import glob

import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data import Field, BucketIterator
from torchtext import data
from collections import Counter
import spacy
import numpy as np
import re
import random
import math
import time
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
authors=['Hemingway','Nietzsche','Wilde']
PATH = "/home/julia/PycharmProjects/seq2seq/books/"
books=[]
nlp = spacy.load('en')

TRAIN_SIZE = 0.8
TEST_SIZE = 0.2

In [3]:
def file_open(path):
    #Remove special characters
    with open(path, encoding='utf-8') as f:
        text = f.read()
        text = re.sub(r'([^a-záéíóúÁÉÍÓÚA-Z0-9,._── :?!¡¿])', '', text)
        return text

In [4]:
def convert_sentences(text):
    file_doc = nlp(text)
    sentences = list(file_doc.sents)
    return sentences

### Abrir Documentos

In [5]:
sents_by_author=[]
for author in authors:
    name_file = PATH + author
    print(name_file)
    books_author = []
    sents = []
    for file in glob.glob(name_file + "/*.txt"):
        books_author.append(file)
        txt = file_open(file)
        sents = sents+convert_sentences(txt)
    sents_by_author.append(sents)
    #print(books_author)
    books.append(books_author)

/home/julia/PycharmProjects/seq2seq/books/Hemingway
/home/julia/PycharmProjects/seq2seq/books/Nietzsche
/home/julia/PycharmProjects/seq2seq/books/Wilde


In [6]:
data_frames={}
d={}

In [7]:
df=["df_{n_author}".format(n_author=a) for a in authors]
df

['df_Hemingway', 'df_Nietzsche', 'df_Wilde']

In [8]:
for i in range(len(authors)):
    d["dataset_dict_{n_author}".format(n_author=authors[i])] = {'input': [sent for sent in sents_by_author[i]], 'output': [sent for sent in sents_by_author[i]]}

In [9]:
df_Hemingway= pd.DataFrame(d["dataset_dict_Hemingway"],columns=["input", 'output'])
df_Nietzsche= pd.DataFrame(d["dataset_dict_Nietzsche"],columns=["input", 'output'])
df_Wilde = pd.DataFrame(d["dataset_dict_Wilde"],columns=["input", 'output'])

In [10]:
data_frames = {df[0]:df_Hemingway,df[1]:df_Nietzsche,df[2]:df_Wilde}

In [11]:
for i in range(len(authors)):
    train_df, test_df = train_test_split(data_frames[df[i]],
                                         test_size=TEST_SIZE,
                                         train_size=TRAIN_SIZE,
                                         random_state=69,
                                         shuffle=True)
    train_df.to_csv(PATH + "/" + authors[i] + "/datasets/trainset_"+authors[i]+".csv", index=False)
    test_df.to_csv(PATH + "/" + authors[i] + "/datasets/testset_"+authors[i]+".csv", index=False)

In [12]:
data_frames[df[0]]

,input,output
0,"(The, UndefeatedManuel, Garcia, climbed, the, ...","(The, UndefeatedManuel, Garcia, climbed, the, ..."
1,"(He, set, down, his, suitcase, and, knocked, o...","(He, set, down, his, suitcase, and, knocked, o..."
2,"(There, was, no, answer, .)","(There, was, no, answer, .)"
3,"(Manuel, ,, standing, in, the, hallway, ,, fel...","(Manuel, ,, standing, in, the, hallway, ,, fel..."
4,"(He, felt, it, through, the, door, .)","(He, felt, it, through, the, door, .)"
...,...,...
24924,"(The, kid, came, out, and, had, to, kill, five...","(The, kid, came, out, and, had, to, kill, five..."
24925,"(He, could, nt, hardly, lift, his, arm, .)","(He, could, nt, hardly, lift, his, arm, .)"
24926,"(He, tried, five, times, and, the, crowd, was,...","(He, tried, five, times, and, the, crowd, was,..."
24927,"(and, then, he, finally, made, it, .)","(and, then, he, finally, made, it, .)"


In [13]:
def tokenize_output(text):
    return [tok.text for tok in nlp.tokenizer(text)]

In [14]:
def tokenize_input(text):
    return [tok.text for tok in nlp.tokenizer(text)][::-1]

In [15]:
SRC = Field(tokenize = tokenize_input,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

In [16]:
TRG = Field(tokenize = tokenize_output,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

In [17]:
fields_n = [('src', SRC), ('trg', TRG)]

In [18]:
train_set, test_set = data.TabularDataset.splits(path=PATH+'/'+authors[0]+'/datasets/',
                                                train = 'trainset_'+authors[0]+'.csv',
                                                validation = 'testset_'+authors[0]+'.csv',
                                                format='csv',
                                                fields=fields_n,
                                                skip_header=True)

In [19]:
print(f'\t\tlen(train_set): {len(train_set)}')
print(f'\t\tlen(dev_set): {len(test_set)}')

		len(train_set): 19943
		len(dev_set): 4986


In [20]:
SRC.build_vocab(train_set,
                         #dev_set,
                         max_size = None,
                         min_freq = 1,
                         vectors = "glove.6B.300d")

In [21]:
TRG.build_vocab(train_set,
                         #dev_set,
                         max_size = None,
                         min_freq = 1,
                         vectors = "glove.6B.300d")

In [22]:
print(f"\tEncoder embedding matrix: {SRC.vocab.vectors.shape}")

	Encoder embedding matrix: torch.Size([15378, 300])


## Creación del modelo 

In [23]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
        #self.embedding.weight.data.copy_(emb_mat)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src len, batch size]

        embedded = self.dropout(self.embedding(src))

        # embedded = [src len, batch size, emb dim]

        outputs, (hidden, cell) = self.rnn(embedded)

        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]

        # outputs are always from the top hidden layer

        return outputs, (hidden, cell)



In [24]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        #self.emb_mat = emb_mat
        self.embedding = nn.Embedding(output_dim, emb_dim)
        #self.embedding.weight.data.copy_(emb_mat)  # load pretrained embedding matrix
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, state):
        (hidden, cell) = state
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]

        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hid dim]
        # context = [n layers, batch size, hid dim]

        input = input.unsqueeze(0)

        # input = [1, batch size]

        embedded = self.dropout(self.embedding(input))

        # embedded = [1, batch size, emb dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        # output = [seq len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]

        # seq len and n directions will always be 1 in the decoder, therefore:
        # output = [1, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0))

        # prediction = [batch size, output dim]

        return prediction, (hidden, cell)


In [25]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src = [src len, batch size]
        # trg = [trg len, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # last hidden state of the encoder is used as the initial hidden state of the decoder
        output,(hidden, cell) = self.encoder(src)

        # first input to the decoder is the <sos> tokens
        input = trg[0, :]

        for t in range(1, trg_len):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, (hidden, cell) = self.decoder(input, (hidden, cell))

            # place predictions in a tensor holding predictions for each token
            outputs[t] = output

            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            # get the highest predicted token from our predictions
            top1 = output.argmax(1)

            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs


## Entrenar modelo 

### Parámetos

In [26]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 4
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
BATCH_SIZE = 32
LEARNING_RATE = 0.001
ENC_EMB_MAT = SRC.vocab.vectors
DEC_EMB_MAT = TRG.vocab.vectors


In [27]:
enc = Encoder(INPUT_DIM, ENC_EMB_DIM,HID_DIM, N_LAYERS, ENC_DROPOUT)

In [28]:
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM ,HID_DIM, N_LAYERS, DEC_DROPOUT)

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
device

device(type='cuda')

In [31]:
train_iterator, test_iterator = BucketIterator.splits(
    (train_set,  test_set),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.src),
    device = device,
    shuffle = True)


In [32]:
model = Seq2Seq(enc, dec, device).to(device)

In [33]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(15378, 256)
    (rnn): LSTM(256, 512, num_layers=4, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(15378, 256)
    (rnn): LSTM(256, 512, num_layers=4, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=15378, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [34]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 31,523,858 trainable parameters


In [35]:
optimizer = optim.Adam(params=model.parameters(),lr = LEARNING_RATE)

In [36]:
TRG_PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

## Entrenamiento de modelo 

In [37]:
def train(model, iterator ,optimizer, criterion, clip):
    model.train()

    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        output = model(src, trg)
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        # trg = [(trg len - 1) * batch size]
        # output = [(trg len - 1) * batch size, output dim]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [38]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            output = model(src, trg, 0)  # turn off teacher forcing
            # trg = [trg len, batch size]
            # output = [trg len, batch size, output dim]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            # trg = [(trg len - 1) * batch size]
            # output = [(trg len - 1) * batch size, output dim]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)


In [39]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [40]:
N_EPOCHS = 10
CLIP = 1
p = 1.0

best_valid_loss = float('inf')

train_loss_epoch = []
test_loss_epoch = []

for epoch in range(N_EPOCHS):
    start_time = time.time()
    #train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    train_loss = train(model=model,
                       iterator=train_iterator,
                       optimizer=optimizer,
                       criterion=criterion,
                       clip=CLIP)
    test_loss = evaluate(model, test_iterator, criterion)
    with torch.no_grad():
        test_loss = train(model=model,
                         iterator=test_iterator,
                         optimizer=None,
                         criterion=criterion,
                         clip=CLIP)
    
    train_loss_epoch.append(train_loss)
    test_loss_epoch.append(test_loss)

    #valid_loss = evaluate(model, test_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if test_loss < best_valid_loss:
        best_valid_loss = test_loss
        torch.save(model.state_dict(), 'tut001-model.pt')


RuntimeError: CUDA out of memory. Tried to allocate 176.00 MiB (GPU 0; 1.95 GiB total capacity; 1.01 GiB already allocated; 63.62 MiB free; 1.09 GiB reserved in total by PyTorch)

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))
test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')